In [1]:
import keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Dropout, BatchNormalization, Dense, Flatten, LayerNormalization
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.applications as appl
from sklearn.model_selection import train_test_split
import os
import cv2
import time
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.metrics as metricsmassey
from tqdm import tqdm
from tensorflow.keras.utils import Sequence
from keras import callbacks  
from keras.models import load_model
from keras.utils import np_utils

In [4]:
load_path='F:/VA/52words_conf_128/'

In [5]:
# model

# model_name1 = 'InceptionResNetV2'
# model_name1 = '4layer'
model_name1 = '8layer2_conf_52words_dslr128-99.34'
#model_name2 = 'VGG16'
# model_name1 = 'DenseNet121'
# model_name1 = 'InceptionV3'

Loading

In [6]:
# #load saved history
# history_const=np.load(load_path+model_name1+'_history.npy',allow_pickle='TRUE').item()

# #load saved model
model1=load_model(load_path+model_name1+'_model.h5')

print("loaded")
model1.summary()

loaded
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 124, 124, 16)      1216      
                                                                 
 dropout (Dropout)           (None, 124, 124, 16)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 120, 120, 32)      12832     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 60, 60, 32)       0         
 )                                                               
                                                                 
 dropout_1 (Dropout)         (None, 60, 60, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 56, 56, 64)        51264     
                                                 

In [7]:
CATEGORIES=np.load(load_path+'cat_52.npy', allow_pickle=True)

In [11]:
## For dslr:
IMG_SIZE = 128
cap = cv2.VideoCapture(0)
org = (20, 20)  
org1 = (310, 20) 
fontScale = 0.65  
font = cv2.FONT_HERSHEY_SIMPLEX
# Blue color in BGR
color = (130, 0, 0)  
# Line thickness of 2 px
thickness = 1 
thickness1 = -1
start_point = (0,0)
end_point = (480,30)
color1 = (255, 255, 255)  


while cap.isOpened():
#image from feeder
    r, img_array = cap.read()
    img_array = cv2.flip(img_array, 1)
    image = img_array[:, 224:800, :]
    image = cv2.resize(image, (480, 480))
    img = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    X = np.array(img).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
    X = X.astype('float32')
    X /= 255
    X = np.array(X)
    Y = model1.predict(X,verbose=0)
    #         print(np.argmax(Y))
    if np.max(Y)>0.2:
        # for display
        image = cv2.rectangle(image, start_point, end_point, color1, thickness1)
        image = cv2.rectangle(image, (0,30), (480,30), color, 2)
        image = cv2.putText(image,"Prediction: "+ CATEGORIES[np.argmax(Y)], org, font, 
               fontScale, color, thickness, cv2.LINE_AA)
        image = cv2.putText(image,"Accuracy: "+ "%.2f" % np.max(Y), org1, font, 
               fontScale, color, thickness, cv2.LINE_AA)
    #             if np.max(Y)>0.95:
        cv2.imwrite('E:/VA/52words_conf/'+CATEGORIES[np.argmax(Y)]+str(np.max(Y))+'.jpg',image)


    cv2.imshow('Realtime testing', image)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
# close the camera
cap.release()

# close all the opened windows
cv2.destroyAllWindows()

# mediapipe testing

# with hands